In [21]:
import findspark
from pyspark.sql import SparkSession
import pyspark.sql.types as T
from delta import *

findspark.init()

In [22]:
builder = SparkSession \
    .builder \
    .config("spark.sql.repl.eagerEval.enabled", "true") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.debug.maxToStringFields", "1000") \
    .appName("notebook")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [27]:
df = spark.read.format("delta").load("./table")

In [28]:
df.orderBy("end_ts").toPandas()

/Users/rafael/personal/spark-playground/stateful-streaming/.venv/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/Users/rafael/personal/spark-playground/stateful-streaming/.venv/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,sensor_id,count,threshold,triggered,start_ts,end_ts,timeout_window,timeout_ms
0,ABC,20,5,True,2023-10-31 11:27:48.456410,2023-10-31 11:28:21.121024,32.664614,30000
1,XYZ,340,2,True,2023-10-31 11:27:37.485385,2023-10-31 11:28:41.028459,63.543074,10000
2,XYZ,1,2,False,2023-10-31 14:07:33.910585,2023-10-31 14:07:52.543673,18.633088,10000
3,XYZ,200,2,True,2023-10-31 14:08:17.746823,2023-10-31 14:08:39.048660,21.301837,10000


In [20]:
# spark.stop()